**Import Library**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import sympy as sp
from PIL import Image, ImageOps
import os

**Size of Data**

In [ ]:
#Size of the image
imgdir = "archive\\s1\\1.pgm"
Face = Image.open(imgdir)
FaceArray = im.pil_to_array(Face)
Height, Width = FaceArray.shape
#Number of classes
num_class = sum(1 for entry in os.scandir("archive") if not entry.is_file())
#Number of files in a class
num_files = sum(1 for entry in os.scandir("archive\\s1") if entry.is_file())
#Total number of files
num_total = num_files*num_class

**Function**

In [133]:
#Process the Image into a flat 1D array
def ProcessImage(img):
    Grayscale = img.convert('L')
    Facematrix = im.pil_to_array(Grayscale)
    FlatFaceArray = np.concat(Facematrix,axis=None)/255

    return FlatFaceArray

#Make the Data Matrix
def Datamatrix(clsdir: str, num_files: int):
    for i in range(num_files):
        imgdir = clsdir +"\\" + str(i+1) + ".pgm"
    #Import image
        Face = Image.open(imgdir)
        FlatFaceArray = ProcessImage(Face)
    #Combine each 1D array in to a matrix
        if i == 0:
            FaceMatrix = FlatFaceArray
        else:
            FaceMatrix = np.column_stack((FaceMatrix,FlatFaceArray))
    return FaceMatrix

#Make a Data Matrix for a class k
def ClassMatrix(k: int):
    clsdir = "archive\\s" + str(k)
    return Datamatrix(clsdir, num_files)
ClassMatrix(2).shape



(10304, 10)

**Make the overall Data Matrix**

In [134]:

for i in range(num_class):
    if i == 0:
        FullMatrix = ClassMatrix(i+1)
    else:
        FullMatrix = np.append(FullMatrix, ClassMatrix(i+1),axis=1)
FullMatrix

array([[0.18823529, 0.23529412, 0.15294118, ..., 0.49019608, 0.46666667,
        0.49019608],
       [0.19215686, 0.23529412, 0.17254902, ..., 0.46666667, 0.47058824,
        0.48627451],
       [0.17647059, 0.24313725, 0.20784314, ..., 0.48627451, 0.47058824,
        0.48627451],
       ...,
       [0.18431373, 0.1254902 , 0.11372549, ..., 0.14117647, 0.34901961,
        0.14117647],
       [0.18039216, 0.13333333, 0.10196078, ..., 0.15294118, 0.36862745,
        0.1372549 ],
       [0.18039216, 0.13333333, 0.11372549, ..., 0.15686275, 0.33333333,
        0.13333333]])

In [135]:
#The average face
MeanFace=np.mean(FullMatrix, axis=1)
ColumnMeanFace= MeanFace[:, np.newaxis]
print(MeanFace)


[0.3357549  0.33559804 0.33696078 ... 0.30145098 0.2975098  0.2950098 ]


**Calculate the eigenvalues and eigenfaces**

In [142]:
#Minus the mean
CenteredFullMatrix=FullMatrix  - ColumnMeanFace
#Find the Eigenface of the Data
BtB=CenteredFullMatrix.T @ CenteredFullMatrix
V, S, Vt = np.linalg.svd(BtB)
#Find the needed amount of eigenvalues
eigsum=np.sum(S)
csum=0
for i in range(S.shape[0]):
    csum +=S[i]
    if csum > 0.9*eigsum:
        e90=i
        break

#S_reduced=S[0:e90]
#S_matrix = np.zeros(np.diag(S).shape)

#S_matrix[:e90, :e90] = np.sqrt(np.diag(S_reduced))
S_matrix = np.sqrt(np.diag(S))
S_inv=np.linalg.inv(S_matrix)
U=CenteredFullMatrix @ V @ S_inv
#Reducing the eigenvalues and vector to the needed amount
U_reduced = U[:,:e90]
S_reduced= np.sqrt(np.diag(S[0:e90]))
U 

array([[ 2.12507923e-03,  1.46851506e-02,  1.99294881e-02, ...,
         7.24583408e-03,  1.97647330e-03,  1.88152095e-09],
       [ 2.11276614e-03,  1.46139383e-02,  2.00092007e-02, ...,
         5.62517084e-04, -2.45204094e-03,  9.40760474e-09],
       [ 2.14250419e-03,  1.46318643e-02,  1.98385174e-02, ...,
        -2.75106052e-03,  3.55081707e-04, -1.74040688e-09],
       ...,
       [ 7.04005567e-03, -1.05610380e-02,  1.41636829e-02, ...,
         4.57939269e-03,  9.57219506e-03, -7.14977960e-09],
       [ 6.39096175e-03, -9.70069545e-03,  1.43943595e-02, ...,
         1.45672324e-04,  9.97750968e-03, -1.59929281e-08],
       [ 7.34479428e-03, -8.81892481e-03,  1.48748947e-02, ...,
         2.13634292e-02,  1.05696646e-02, -1.09128215e-08]])

In [143]:
#Reconstruct an example Eigenface as a picture
Eigenface=U_reduced 
face = Eigenface[:,1]*255
face = np.split(face, Height)
for i in range(len(face)):
    if i == 0:
        recface=face[0]
    else:
        recface = np.vstack((recface,face[i]))
recim = Image.fromarray(recface).convert("L")
recim.save('test.png','PNG')

In [152]:
#Projecting a vector in to the facespace
def Facespace(vector):
    Omega = np.array([])
    for i in range(e90):
        component=U_reduced[:,i] @ vector
        Omega = np.append(Omega,[[component]] )
    return Omega
#Omega= Omega[:, np.newaxis]

#the average of a class k in facespace
def avgClassOmega(k: int):
    S = 0
    for i in range(num_files):
        S+= Facespace(ClassMatrix(k)[:,i])
    return S/len(range(num_files))

#Find the threshold for the picture to be indentified as a face in class k
def Classepsilon(k: int):
    Cepsilon = 0
    for i in range(num_files):
        e=np.linalg.norm(Facespace(ClassMatrix(k)[:,i])-avgClassOmega(k))
        if e > Cepsilon:
            Cepsilon = e
    return Cepsilon*1.2


In [150]:

#The average of all the data in facespace
S = 0
for i in range(num_total):
    S+= Facespace(CenteredFullMatrix[:,i])
avgOmega = S/len(range(num_files))
#Find the threshold for the picture to be identified as a face, let that be epsilon
epsilon=0

for i in range(num_total):
    e = np.linalg.norm(Facespace(CenteredFullMatrix[:,i])-avgOmega)
    if e > epsilon:
        epsilon = e
#Let the threshold be 1.2epsilon
epsilon = 1.2*epsilon
epsilon

np.float64(25.77179594469942)

In [ ]:

#Input = ProcessImage(Image.open("32.JPG"))-MeanFace
#Input.shape
#Let the image that need to be specify is input
image=Image.open("9.pgm")
Omega = Facespace(ProcessImage(image)-MeanFace)
epsilon > np.linalg.norm(Facespace(CenteredFullMatrix[:,2])-avgOmega)

np.True_

In [147]:
Height, Width

(112, 92)

np.float64(12.7453321461929)